<a href="https://colab.research.google.com/github/mostafa-asefy/neural-networks-from-zero/blob/main/Artificial_Neural_Networks_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Network Class Definition

In [6]:
class NeuralNetwork:

    # initialize the neural network 
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # set number of nodes in input, hidden and output layers
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        # set the learning rate
        self.learning_rate = learning_rate
        pass

    # train the neural network
    def train():
        pass

    # query the neural network
    def query():
        pass

# Neural Network Instance Creation

In [10]:
# number of nodes in each layer
input_nodes = 3
hidden_nodes = 3
output_nodes = 3

# learning rate
learning_rate = 0.3

# create intance of neural network
nn = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)
print(nn)